In [628]:
import pandas as pd
import geopandas as gpd
import re
import textdistance
import numpy as np
import math

In [629]:
elevator_list = pd.read_csv('../data/raw/EE_master_list.csv')
stations = gpd.read_file('../data/raw/subway_stations.geojson')
turnstile_remotes = pd.read_excel('../data/raw/Remote-Booth-Station.xls')
gtfs = pd.read_csv('../data/raw/google_transit/stops.txt')

In [630]:
elevator_list.head()

,station_name,equipment_id,description,borough,subway_lines,equipment_type,ada_compliant,is_active,direction,subset_lines,connection_to_street
0,125 St,EL125,125 St & Lexington Ave (NE corner) to mezzanin...,MN,4/5/6,EL,Y,Y,both directions,4/5/6,Y
1,125 St,EL142,125 St & St Nicholas Ave (SW corner) to mezzan...,MN,A/B/C/D,EL,Y,Y,both directions,A/B/C/D,Y
2,125 St,EL143,mezzanine to downtown platform,MN,A/B/C/D,EL,Y,Y,downtown,A/B/C/D,N
3,125 St,EL126,mezzanine to platforms for service in both dir...,MN,4/5/6,EL,Y,Y,both directions,4/5/6,N
4,125 St,EL144,mezzanine to uptown platform,MN,A/B/C/D,EL,Y,Y,uptown,A/B/C/D,N


In [631]:
stations.head()

,name,url,line,objectid,notes,geometry
0,Astor Pl,http://web.mta.info/nyct/service/,4-6-6 Express,1,"4 nights, 6-all times, 6 Express-weekdays AM s...",POINT (-73.99107 40.73005)
1,Canal St,http://web.mta.info/nyct/service/,4-6-6 Express,2,"4 nights, 6-all times, 6 Express-weekdays AM s...",POINT (-74.00019 40.71880)
2,50th St,http://web.mta.info/nyct/service/,1-2,3,"1-all times, 2-nights",POINT (-73.98385 40.76173)
3,Bergen St,http://web.mta.info/nyct/service/,2-3-4,4,"4-nights, 3-all other times, 2-all times",POINT (-73.97500 40.68086)
4,Pennsylvania Ave,http://web.mta.info/nyct/service/,3-4,5,"4-nights, 3-all other times",POINT (-73.89489 40.66471)


In [632]:
turnstile_remotes.head()

,Remote,Booth,Station,Line Name,Division
0,R001,A060,WHITEHALL ST,R1,BMT
1,R001,A058,WHITEHALL ST,R1,BMT
2,R001,R101S,SOUTH FERRY,R1,IRT
3,R002,A077,FULTON ST,ACJZ2345,BMT
4,R002,A081,FULTON ST,ACJZ2345,BMT


In [633]:
gtfs = gtfs[gtfs.location_type == 1]
gtfs.head()

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
0,101,NaN,Van Cortlandt Park - 242 St,NaN,40.889248,-73.898583,NaN,NaN,1,NaN
3,103,NaN,238 St,NaN,40.884667,-73.900870,NaN,NaN,1,NaN
6,104,NaN,231 St,NaN,40.878856,-73.904834,NaN,NaN,1,NaN
9,106,NaN,Marble Hill - 225 St,NaN,40.874561,-73.909831,NaN,NaN,1,NaN
12,107,NaN,215 St,NaN,40.869444,-73.915279,NaN,NaN,1,NaN


In [634]:
gtfs_routes = pd.read_csv('../data/raw/google_transit/routes.txt')
gtfs_trips = pd.read_csv('../data/raw/google_transit/trips.txt')
gtfs_stop_times = pd.read_csv('../data/raw/google_transit/stop_times.txt')

In [635]:
gtfs_stop_times = gtfs_stop_times[gtfs_stop_times.trip_id.str.contains('Weekday')]
gtfs_lines = gtfs_stop_times.merge(gtfs_trips,on="trip_id")
gtfs_lines = gtfs_lines.merge(gtfs_routes,on='route_id')

In [637]:
gtfs_lines['stop_id'] = [re.sub('N$|S$','',x) for x in gtfs_lines.stop_id]

gtfs_lines['lines'] = gtfs_lines[['stop_id','route_short_name']].groupby(['stop_id'])['route_short_name'].transform(lambda x: ','.join(x.unique()))

In [638]:
gtfs_lines = gtfs_lines[['stop_id','lines']]
gtfs_lines = gtfs_lines.drop_duplicates()

In [639]:
gtfs_lines.head()

,stop_id,lines
0,101,1
1,103,1
2,104,1
3,106,1
4,107,1


In [641]:
gtfs = gtfs.merge(gtfs_lines[['stop_id','lines']],how='left',on='stop_id')

gtfs = gtfs[~gtfs.lines.isnull()]

In [642]:
gtfs.lines.unique()

array(['1', '1,2,3', '1,2', '2', '2,5', '2,5,5X', '2,3', '2,3,4,5,5X',
       '2,3,4', '2,3,4,5', '3', '4', '4,5,5X', '5,5X', '6,6X', '6',
       '4,5,5X,6,6X', '4,6,6X', '7,7X', '7', 'S', 'A', 'A,C', 'A,B,C',
       'A,B,C,D', 'A,C,E', 'A,C,F,FX', 'A,C,G', 'F,FX', 'F,FX,M,N,Q,R',
       'F,FX,M', 'D', 'B,D', 'B,D,E', 'B,D,F,FX,M', 'B,Q', 'B,S,Q', 'Q',
       'F,FX,Q', 'D,F,FX,N,Q', 'E', 'E,F,FX', 'E,M', 'F,G', 'F,FX,G',
       'E,J,Z', 'E,F,FX,M,R', 'E,M,R', 'G', 'A,S', 'J,Z', 'J', 'L', 'M',
       'J,Z,M', 'J,M', 'N,W,Q', 'N,Q', 'M,N,Q,R', 'N,W', 'N,W,R',
       'N,W,Q,R', 'B,D,N,W,Q,R', 'D,N,W,Q,R', 'D,N,W,R', 'R', 'SIR'],
      dtype=object)

Example of different naming conventions

In [643]:
stations[stations.name.str.contains('Time')].name.unique()[0]

'Times Sq - 42nd St'

In [644]:
elevator_list[elevator_list.station_name.str.contains('Time')].station_name.unique()[0]

'Times Sq - 42 St'

In [645]:
turnstile_remotes[turnstile_remotes.Station.str.contains('TIME')].Station.unique()[0]

'42 ST-TIMES SQ'

In [646]:
gtfs[gtfs.stop_name.str.contains('Time')].stop_name.unique()[0]

'Times Sq - 42 St'

### Standardization

In [647]:
stations = pd.DataFrame(stations.drop('geometry',axis=1))

In [648]:
def make_ordinal(s):
    ordinal = lambda n: "%d%s" % (n,"tsnrhtdd"[(math.floor(n/10)%10!=1)*(n%10<4)*n%10::4])
    
    name_ord = []
    for x in s:
        x = x.title()
        m = re.findall(r'\d+', x)
        if(len(m) > 0):
            num = m[0]
            t = re.sub('{}'.format(num), ordinal(int(num)), x)
            name_ord.append(t)
        else:
            t = x
            name_ord.append(t)
    return name_ord

Standardizing station names

In [649]:
stations['name_ord'] = stations.name
turnstile_remotes['name_ord'] = make_ordinal(turnstile_remotes.Station)
elevator_list['name_ord'] = make_ordinal(elevator_list.station_name)
gtfs['name_ord'] = make_ordinal(gtfs.stop_name)

Standardizing lines

In [650]:
stations["clean_lines"] = [re.sub('-','',re.sub('-\d+ Express','',x)) for x in stations.line]
turnstile_remotes['clean_lines'] = [re.sub('-','',re.sub(r'(\w)(?!$)',r'\1-',str(x))) for x in turnstile_remotes['Line Name']]
elevator_list['clean_lines'] = [re.sub('-','',re.sub('/', '-',re.sub('(/METRO-NORTH)|(/LIRR)','', x))) for x in elevator_list.subway_lines]
gtfs['clean_lines'] = [re.sub('-','',re.sub(',','-',re.sub(',((\d)|(\w))X','',x))) for x in gtfs.lines]

Dropping unnecessary columns

In [651]:
stations = stations[['name','name_ord','clean_lines']]
elevator_list = elevator_list[['equipment_id','station_name','name_ord','clean_lines']]
turnstile_remotes = turnstile_remotes[['Remote','Station','name_ord','clean_lines']]
gtfs = gtfs[['stop_id','stop_name','stop_lat','stop_lon','name_ord','clean_lines']]

### Text-Matching Station Names

In [652]:
# station_name, el_station_name, elevators, turnstile_station_name, turnstile_remotes, gtfs_station_name, gtfs_stop_id, lat, lon

In [653]:
# def match_jaccard(ee, platforms):
#     for index, row in ee.iterrows():
#         if row.possible_stops == '':
#             subset = platforms[platforms.routes_wkd.str.contains(row.line)]
#             if subset.shape[0] > 0:
#                 subset_stop_names = pd.DataFrame(subset.stop_name.unique(),columns=['stop_name'])
#                 name_dist = [textdistance.jaccard(row.station_name,y) for y in subset_stop_names.stop_name]

#                 matched_station_name = subset_stop_names.iloc[np.argmax(name_dist),0]
#                 matched_stop_ids = subset[subset.stop_name == matched_station_name][['stop_id']]
#                 score = max(name_dist)
#                 if score >=0.8:
#                     ee.loc[index,'possible_stops'] = ', '.join(matched_stop_ids.stop_id)
#     return ee

# def match_jaro_winkler(ee, platforms):
#     for index, row in ee.iterrows():
#         if row.possible_stops == '':
#             subset = platforms[platforms.routes_wkd.str.contains(row.line)]
#             if subset.shape[0] > 0:
#                 subset_stop_names = pd.DataFrame(subset.stop_name.unique(),columns=['stop_name'])
#                 name_dist = [textdistance.jaro_winkler(row.station_name,y) for y in subset_stop_names.stop_name]

#                 matched_station_name = subset_stop_names.iloc[np.argmax(name_dist),0]
#                 matched_stop_ids = subset[subset.stop_name == matched_station_name][['stop_id']]
#                 score = max(name_dist)
#                 if score > 0.79:
#                     ee.loc[index,'possible_stops'] = ', '.join(matched_stop_ids.stop_id)
#     return ee

In [654]:
stations.head()

,name,name_ord,clean_lines
0,Astor Pl,Astor Pl,46
1,Canal St,Canal St,46
2,50th St,50th St,12
3,Bergen St,Bergen St,234
4,Pennsylvania Ave,Pennsylvania Ave,34


In [655]:
# el_unique = elevator_list[['station_name','name_ord','clean_lines']]
# el_unique = el_unique.drop_duplicates()
elevator_list.reset_index(drop=True,inplace=True)
elevator_list['station_match'] = ''
elevator_list['station_lines'] = ''
for i,row in elevator_list.iterrows():
    ## station matching lines
    st_line_matches = [y if len(textdistance.lcsstr(row.clean_lines,y)) > 0 else None for y in stations.clean_lines]
    st_line_matches = [x for x in st_line_matches if x is not None]
    st_subset = stations[stations.clean_lines.isin(st_line_matches)] 
    
    ## Fails to find the right match for just 59th St
    if row.station_name == '59 St':
        continue
    
    ## elevator
    if st_subset.shape[0] > 0:
        st_dist = [textdistance.jaccard(row.name_ord,y) for y in st_subset.name_ord]
        st_match = st_subset.iloc[np.argmax(st_dist),]
        st_score = max(st_dist)
        if st_score > 0.75:
            elevator_list.iloc[i,][['station_match','station_lines']] = st_match[['name_ord','clean_lines']]
#             print(row['station_name'],": ST ", st_match," (",st_score,")")
        else:
            st_dist = [textdistance.jaro_winkler(row.name_ord,y) for y in st_subset.name_ord]
            st_match = st_subset.iloc[np.argmax(st_dist),]
            st_score = max(st_dist)
            elevator_list.iloc[i,][['station_match','station_lines']] = st_match[['name_ord','clean_lines']]
#             print(row['station_name'],": ST ", st_match," (",st_score,")")

In [656]:
## Manual overrides
elevator_list.loc[(elevator_list.station_name == '57 St - 7 Av')&(elevator_list.station_match == ''),['clean_lines','station_match','station_lines']] = ['NQRW','57th St','NQRW']
elevator_list.loc[(elevator_list.station_name == '59 St')&(elevator_list.station_match == ''),['clean_lines','station_match','station_lines']] = ['456','Lexington Ave - 59th St','456']
elevator_list.loc[(elevator_list.station_name == '68 St / Hunter College')&(elevator_list.station_match == ''),['clean_lines','station_match','station_lines']] = ['46','68th St - Hunter College','46']
elevator_list.loc[(elevator_list.station_name == '86 St')&(elevator_list.station_match == ''),['clean_lines','station_match','station_lines']] = ['456','86th St','456']
elevator_list.loc[(elevator_list.station_name == 'Bedford Park Blvd/Grand Concourse Line')&(elevator_list.station_match == ''),['clean_lines','station_match','station_lines']] = ['BD','Bedford Park Blvd','BD']
elevator_list.loc[(elevator_list.station_name == 'Chambers St')&(elevator_list.station_match == ''),['clean_lines','station_match','station_lines']] = ['JZ','Chambers St','JZ']

In [657]:
elevator_list.head()

,equipment_id,station_name,name_ord,clean_lines,station_match,station_lines
0,EL125,125 St,125th St,456,125th St,456
1,EL142,125 St,125th St,ABCD,125th St,ABCD
2,EL143,125 St,125th St,ABCD,125th St,ABCD
3,EL126,125 St,125th St,456,125th St,456
4,EL144,125 St,125th St,ABCD,125th St,ABCD


In [658]:
# elevator_list['equipments'] = 
el_station_merge = elevator_list.copy()
el_station_merge['equipments'] = el_station_merge.groupby(['station_match','station_lines'])['equipment_id'].transform(lambda x : ','.join(x.unique()))

In [660]:
el_station_merge.drop(['equipment_id','name_ord'],axis=1,inplace=True)
el_station_merge = el_station_merge.drop_duplicates()

In [664]:
crosswalk = stations.merge(el_station_merge,how='left',left_on=['name','clean_lines'],right_on=['station_match','station_lines'])
crosswalk.rename(columns={'clean_lines_x':'clean_lines','station_name':'el_station_name','clean_lines_y':'el_lines'},inplace=True)
crosswalk.drop(['station_match','station_lines'],axis=1,inplace=True)
crosswalk.fillna('',inplace=True)

In [665]:
crosswalk.head()

,name,name_ord,clean_lines,el_station_name,el_lines,equipments
0,Astor Pl,Astor Pl,46,,,
1,Canal St,Canal St,46,Canal St,6,"EL324,EL325"
2,50th St,50th St,12,,,
3,Bergen St,Bergen St,234,,,
4,Pennsylvania Ave,Pennsylvania Ave,34,,,


In [666]:
gtfs.head()

,stop_id,stop_name,stop_lat,stop_lon,name_ord,clean_lines
0,101,Van Cortlandt Park - 242 St,40.889248,-73.898583,Van Cortlandt Park - 242nd St,1
1,103,238 St,40.884667,-73.900870,238th St,1
2,104,231 St,40.878856,-73.904834,231st St,1
3,106,Marble Hill - 225 St,40.874561,-73.909831,Marble Hill - 225th St,1
4,107,215 St,40.869444,-73.915279,215th St,1


In [667]:
crosswalk.reset_index(drop=True,inplace=True)
crosswalk['gtfs_station_name'] = ''
crosswalk['gtfs_lines'] = ''

for i,row in crosswalk.iterrows():

#     ## turnstile matching lines
#     ts_line_matches = [y if len(textdistance.lcsstr(row.clean_lines,y)) > 0 else None for y in turnstile_remotes.clean_lines]
#     ts_line_matches = [x for x in ts_line_matches if x is not None]
#     ts_subset = turnstile_remotes[turnstile_remotes.clean_lines.isin(ts_line_matches)]

    ## gtfs matching lines
    gtfs_line_matches = [y if len(textdistance.lcsstr(row.clean_lines,y)) > 0 else None for y in gtfs.clean_lines]
    gtfs_line_matches = [x for x in gtfs_line_matches if x is not None]
    gtfs_subset = gtfs[gtfs.clean_lines.isin(gtfs_line_matches)]
    
    ###### distances
    
    ## turnstile
#     if ts_subset.shape[0] > 0:
#         ts_dist = [textdistance.jaccard(row.name_ord,y) for y in ts_subset.name_ord]
#         ts_match = ts_subset.iloc[np.argmax(ts_dist),1]
#         ts_score = max(ts_dist)
#         print(row['name'],": TS ", ts_match," (",ts_score,")")

    ## gtfs
    
    ## exceptions where it fails
    if((row.name_ord == '46th St') | (row.name_ord == '57th St')):
        continue
        
    if gtfs_subset.shape[0] > 0:
        gtfs_dist = [textdistance.jaccard(row.name_ord,y) for y in gtfs_subset.name_ord]
        gtfs_match = gtfs_subset.iloc[np.argmax(gtfs_dist),]
        gtfs_score = max(gtfs_dist)
        if gtfs_score > 0.88:
            crosswalk.iloc[i,][['gtfs_station_name','gtfs_lines']] = gtfs_match[['stop_name','clean_lines']]
#             print(row['name'],": EL ", gtfs_match," (",gtfs_score,")")
        else:
            gtfs_dist = [textdistance.jaro_winkler(row.name_ord,y) for y in gtfs_subset.name_ord]
            gtfs_match = gtfs_subset.iloc[np.argmax(gtfs_dist),]
            gtfs_score = max(gtfs_dist)
            if gtfs_score > 0.74:
                crosswalk.iloc[i,][['gtfs_station_name','gtfs_lines']] = gtfs_match[['stop_name','clean_lines']]
#                 print(row['name'],": EL ", gtfs_match," (",gtfs_score,")")



In [668]:
crosswalk.head()

,name,name_ord,clean_lines,el_station_name,el_lines,equipments,gtfs_station_name,gtfs_lines
0,Astor Pl,Astor Pl,46,,,,Astor Pl,46
1,Canal St,Canal St,46,Canal St,6,"EL324,EL325",Canal St,46
2,50th St,50th St,12,,,,50 St,12
3,Bergen St,Bergen St,234,,,,Bergen St,234
4,Pennsylvania Ave,Pennsylvania Ave,34,,,,Pennsylvania Av,2345


In [686]:
## Manual overrides
crosswalk.loc[(crosswalk.name_ord == 'Lexington Ave - 59th St')&(crosswalk.gtfs_station_name == ''),['gtfs_station_name','gtfs_lines']] = ['59 St','456']
crosswalk.loc[(crosswalk.name_ord == 'Long Island City - Court Sq')&(crosswalk.gtfs_station_name == ''),['gtfs_station_name','gtfs_lines']] = ['Court Sq - 23 St','G']
crosswalk.loc[(crosswalk.name_ord == '46th St')&(crosswalk.clean_lines=='EMR')&(crosswalk.gtfs_station_name == ''),['gtfs_station_name','gtfs_lines']] = ['46 St','EMR']
crosswalk.loc[(crosswalk.name_ord == '46th St')&(crosswalk.clean_lines=='7')&(crosswalk.gtfs_station_name == ''),['gtfs_station_name','gtfs_lines']] = ['46 St - Bliss St','7']
crosswalk.loc[(crosswalk.name_ord == 'Gravesend - 86th St')&(crosswalk.gtfs_station_name == ''),['gtfs_station_name','gtfs_lines']] = ['86 St','NWQ']
crosswalk.loc[(crosswalk.name_ord == 'Lower East Side - 2nd Ave')&(crosswalk.gtfs_station_name == ''),['gtfs_station_name','gtfs_lines']] = ['2 Av','F']
crosswalk.loc[(crosswalk.name_ord == '57th St')&(crosswalk.clean_lines=='F')&(crosswalk.gtfs_station_name == ''),['gtfs_station_name','gtfs_lines']] = ['57 St','FM']
crosswalk.loc[(crosswalk.name_ord == '57th St')&(crosswalk.clean_lines=='NQRW')&(crosswalk.gtfs_station_name == ''),['gtfs_station_name','gtfs_lines']] = ['57 St - 7 Av','NWQR']

In [693]:
turnstile_remotes.head()

,Remote,Station,name_ord,clean_lines
0,R001,WHITEHALL ST,Whitehall St,R1
1,R001,WHITEHALL ST,Whitehall St,R1
2,R001,SOUTH FERRY,South Ferry,R1
3,R002,FULTON ST,Fulton St,ACJZ2345
4,R002,FULTON ST,Fulton St,ACJZ2345


In [691]:
stations_w_issues = ['36th Ave','111th St','168th St','104th St','7th Ave','28th St','39th Ave','81st St','30th Ave', 'Broadway Junction',
'49th St', '57th St', '80th St','96th St','176th St']

In [811]:
crosswalk.reset_index(drop=True,inplace=True)
crosswalk['turnstile_station_name'] = ''
crosswalk['turnstile_lines'] = ''

for i,row in crosswalk.iterrows():

    ## turnstile matching lines
    ts_line_matches = [y if len(textdistance.lcsstr(row.clean_lines,y)) > 0 else None for y in turnstile_remotes.clean_lines]
    ts_line_matches = [x for x in ts_line_matches if x is not None]
    ts_subset = turnstile_remotes[turnstile_remotes.clean_lines.isin(ts_line_matches)]

    
    ##### distances
    if (row.name_ord in stations_w_issues):
        continue
        
    # turnstile
    if ts_subset.shape[0] > 0:
        ts_dist = [textdistance.jaccard(row.name_ord,y) for y in ts_subset.name_ord]
        ts_match = ts_subset.iloc[np.argmax(ts_dist),]
        ts_score = max(ts_dist)
        if ts_score > 0.88:
            crosswalk.iloc[i,][['turnstile_station_name','turnstile_lines']] = ts_match[['Station','clean_lines']]
#             print(row['name'],": TS ", ts_match," (",ts_score,")")
        else:
            ts_dist = [textdistance.jaro_winkler(row.name_ord,y) for y in ts_subset.name_ord]
            ts_match = ts_subset.iloc[np.argmax(ts_dist),]
            ts_score = max(ts_dist)
            if ts_score > 0.81:
                crosswalk.iloc[i,][['turnstile_station_name','turnstile_lines']] = ts_match[['Station','clean_lines']]
#                 print(row['name'],": TS ", ts_match," (",ts_score,")")



In [814]:
missing_vals = crosswalk[crosswalk.turnstile_station_name == ''][['name','clean_lines']]
missing_vals.reset_index(drop=True,inplace=True)

In [815]:
## manual overrides
ts_override = [['MAIN ST','7'],['138 ST-3 AVE','6'],['42 ST-GRD CNTRL','4567S'],['96 ST','6'],['61 ST/WOODSIDE','7'],['96 ST','BC'],
['168 ST-BROADWAY','1AC'],['UNION TPK-KEW G','EF'],['WASHINGTON-36 A','NQ'],['42 ST-GRD CNTRL','4567S'],['GREENWOOD-111','A'],
['OXFORD-104 ST','A'],['7 AV-PARK SLOPE','FG'],['7 AVE','BQ'],['FLATBUSH AVE','25'],['28 ST-BROADWAY','NR'],['COURT SQ','EMG'],
['VAN ALSTON-21ST','G'],['BEEBE-39 AVE','NQ'],['96 ST','123'],['110 ST-CPN','23'],['81 ST-MUSEUM','BC'],['110 ST-CATHEDRL','1'],['168 ST-BROADWAY','1AC'],['111 ST','7'],
['LEFFERTS BLVD','A'],['28 ST','1'],['28 ST','6'],['42 ST-GRD CNTRL','4567S'],['FOREST PARKWAY','J'],['111 ST','J'],['MYRTLE AVE','LM'],               
['ROCKAWAY PKY','L'],['EAST 105 ST','L'],['BROADWAY-ENY','ACJLZ'],['ELDERTS LANE','JZ'],['MYRTLE AVE','LM'],['VAN WYCK BLVD','EF'],
['HOYT ST-ASTORIA','NQ'],['DITMARS BL-31 S','NQ'],['148 ST-LENOX','3'],['242 ST','1'],['E TREMONT AVE','25'],['DYRE AVE','5'],
['BROADWAY-ENY','ACJLZ'],['149 ST-3 AVE','25'],['GRAND-30 AVE','NQ'],['NEW UTRECHT AVE','ND'],['86 ST','N'],['22 AVE-BAY PKY','F'],
['7 AVE-53 ST','BDE'],['57 ST','F'],['49 ST-7 AVE','NQR'],['57 ST-7 AVE','NQR'],['2 AVE','F'],['BOROUGH HALL/CT','2345R'],['BROADWAY-ENY','ACJLZ'],
['BROOKLYN BRIDGE','456JZ'],['METROPOLITAN AV','M'],['ROOSEVELT AVE','EFMR7'],['E 177 ST-PARKCH','6'],['HUDSON-80 ST','A'],['STILLWELL AVE','DFNQ'],['',''],
['',''],['',''],['','']]


In [816]:
turnstile_override = pd.DataFrame(ts_override)
turnstile_override.rename(columns={0:'turnstile_station_name',1:'turnstile_lines'},inplace=True)

In [817]:
turnstile_override = pd.concat([missing_vals,turnstile_override],axis=1)

In [819]:
turnstile_override.head()

,name,clean_lines,turnstile_station_name,turnstile_lines
0,Flushing - Main St,7,MAIN ST,7
1,3rd Ave - 138th St,6,138 ST-3 AVE,6
2,Grand Central - 42nd St,456,42 ST-GRD CNTRL,4567S
3,96th St,46,96 ST,6
4,Woodside - 61st St,7,61 ST/WOODSIDE,7


In [818]:
crosswalk.head()

,name,name_ord,clean_lines,el_station_name,el_lines,equipments,gtfs_station_name,gtfs_lines,turnstile_station_name,turnstile_lines
0,Astor Pl,Astor Pl,46,,,,Astor Pl,46,ASTOR PLACE,6
1,Canal St,Canal St,46,Canal St,6,"EL324,EL325",Canal St,46,CANAL ST,JNQRZ6
2,50th St,50th St,12,,,,50 St,12,50 ST,1
3,Bergen St,Bergen St,234,,,,Bergen St,234,BERGEN ST,23
4,Pennsylvania Ave,Pennsylvania Ave,34,,,,Pennsylvania Av,2345,PENNSYLVANIA AV,3


In [832]:
for i,row in crosswalk.iterrows():
    if (row.turnstile_station_name == ''):
        ts_match = turnstile_override[(turnstile_override.name == row.name_ord)&(turnstile_override.clean_lines == row.clean_lines)][['turnstile_station_name','turnstile_lines']]
        crosswalk.iloc[i,][['turnstile_station_name','turnstile_lines']] = ts_match.values[0]

In [834]:
crosswalk.head()

,name,name_ord,clean_lines,el_station_name,el_lines,equipments,gtfs_station_name,gtfs_lines,turnstile_station_name,turnstile_lines
0,Astor Pl,Astor Pl,46,,,,Astor Pl,46,ASTOR PLACE,6
1,Canal St,Canal St,46,Canal St,6,"EL324,EL325",Canal St,46,CANAL ST,JNQRZ6
2,50th St,50th St,12,,,,50 St,12,50 ST,1
3,Bergen St,Bergen St,234,,,,Bergen St,234,BERGEN ST,23
4,Pennsylvania Ave,Pennsylvania Ave,34,,,,Pennsylvania Av,2345,PENNSYLVANIA AV,3


In [836]:
crosswalk.drop('name_ord',axis=1,inplace=True)
crosswalk.rename(columns={'clean_lines':'lines'},inplace=True)

In [837]:
crosswalk.head()

,name,lines,el_station_name,el_lines,equipments,gtfs_station_name,gtfs_lines,turnstile_station_name,turnstile_lines
0,Astor Pl,46,,,,Astor Pl,46,ASTOR PLACE,6
1,Canal St,46,Canal St,6,"EL324,EL325",Canal St,46,CANAL ST,JNQRZ6
2,50th St,12,,,,50 St,12,50 ST,1
3,Bergen St,234,,,,Bergen St,234,BERGEN ST,23
4,Pennsylvania Ave,34,,,,Pennsylvania Av,2345,PENNSYLVANIA AV,3


In [839]:
crosswalk = crosswalk.merge(gtfs.drop('name_ord',axis=1),how='left',left_on=['gtfs_station_name','gtfs_lines'],right_on=['stop_name','clean_lines'])

In [840]:
crosswalk.head()

,name,lines,el_station_name,el_lines,equipments,gtfs_station_name,gtfs_lines,turnstile_station_name,turnstile_lines,stop_id,stop_name,stop_lat,stop_lon,clean_lines
0,Astor Pl,46,,,,Astor Pl,46,ASTOR PLACE,6,636,Astor Pl,40.730054,-73.991070,46
1,Canal St,46,Canal St,6,"EL324,EL325",Canal St,46,CANAL ST,JNQRZ6,639,Canal St,40.718803,-74.000193,46
2,50th St,12,,,,50 St,12,50 ST,1,126,50 St,40.761728,-73.983849,12
3,Bergen St,234,,,,Bergen St,234,BERGEN ST,23,236,Bergen St,40.680829,-73.975098,234
4,Pennsylvania Ave,34,,,,Pennsylvania Av,2345,PENNSYLVANIA AV,3,255,Pennsylvania Av,40.664635,-73.894895,2345


In [843]:
crosswalk.drop(['stop_name','clean_lines'],axis=1,inplace=True)
crosswalk.rename(columns={'stop_id':'gtfs_stop_id','stop_lat':'lat','stop_lon':'lon'},inplace=True)

In [846]:
turnstile_remotes['turnstile_remotes'] = turnstile_remotes.groupby(['Station','clean_lines'])['Remote'].transform(lambda x : ','.join(x.unique()))

In [850]:
turnstile_merge = turnstile_remotes.drop(['Remote','name_ord'],axis=1).drop_duplicates()

In [851]:
turnstile_merge.head()

,Station,clean_lines,turnstile_remotes
0,WHITEHALL ST,R1,R001
2,SOUTH FERRY,R1,R001
3,FULTON ST,ACJZ2345,"R002,R014"
6,CYPRESS HILLS,J,R003
7,ELDERTS LANE,JZ,R004


In [854]:
crosswalk = crosswalk.merge(turnstile_merge,how='left',left_on=['turnstile_station_name','turnstile_lines'],right_on=['Station','clean_lines']).drop(['Station','clean_lines'],axis=1)

In [855]:
crosswalk.to_csv('../data/crosswalk/Master_crosswalk.csv',index=False)